# Mongolia's Political Ad Spending April 2021

This is a monthly template for making the political ad spending report for Mongolia. The following visualization/analyses will be performed:

1. Number of ads by day
    - Use days active to determine this
2. Top Facebook pages by ad spending
3. Total amount spent during this time (show a range)
4. Show ads with most impressions - top 5
5. LDA analysis for top 3 topics

In [1]:
import pandas as pd
import numpy as np
import psycopg2
import ast
import datetime

from config import conn_str
conn_str = conn_str('facebook')

In [2]:
conn = psycopg2.connect(conn_str)

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [4]:
start_date = datetime.datetime(year=2021, month=4, day=1)
end_date = datetime.datetime(year=2021, month=4, day=30)

## Import Tables

There are two tables:
- fb_ad_lib_ad_data
- fb_ad_lib_pages

In [5]:
ads = pd.read_sql('select * from fb_ad_lib_ad_data;', conn)
pages = pd.read_sql('select * from fb_ad_lib_pages;', conn)

In [6]:
ads.shape

(5555, 19)

In [7]:
pages.shape

(1443, 5)

In [8]:
ads.head()

,ad_creation_time,ad_creative_body,ad_creative_link_title,ad_delivery_start_time,ad_delivery_stop_time,ad_snapshot_url,currency,demographic_distribution,funding_entity,id,impressions,page_id,page_name,potential_reach,publisher_platforms,region_distribution,spend,ad_creative_link_caption,ad_creative_link_description
0,2021-04-30,Монгол улсын ерөнхийлөгчид Ардчилсан намаас нэ...,Монгол улсын ерөнхийлөгчид Ардчилсан намаас нэ...,2021-04-30,2021-05-02,https://www.facebook.com/ads/archive/render_ad...,USD,"[{'percentage': '0.046576', 'age': '55-64', 'g...",Enkhmurun Erdenebat,808086169839504,"{'lower_bound': '30000', 'upper_bound': '34999'}",100179472105159,Democratic Party Ардчилсан нам,{'lower_bound': '1000001'},['facebook'],"[{'percentage': '0.014488', 'region': 'Övörkha...","{'lower_bound': '0', 'upper_bound': '99'}",nan,nan
1,2021-04-29,САНАЛ ӨГӨХ ЗААВАР:🇲🇳nАрдчилсан нам Монгол улса...,nan,2021-04-30,2021-05-02,https://www.facebook.com/ads/archive/render_ad...,USD,"[{'percentage': '0.000132', 'age': '25-34', 'g...",Enkhmurun Erdenebat,811746283053829,"{'lower_bound': '25000', 'upper_bound': '29999'}",100179472105159,Democratic Party Ардчилсан нам,{'lower_bound': '1000001'},"['facebook', 'instagram']","[{'percentage': '0.012833', 'region': 'Arkhang...","{'lower_bound': '0', 'upper_bound': '99'}",nan,nan
2,2021-04-29,🇲🇳Монгол улсын #Ерөнxийлөгчид Ардчилсан намаас...,nan,2021-04-30,2021-05-01,https://www.facebook.com/ads/archive/render_ad...,USD,"[{'percentage': '6.1E-5', 'age': '55-64', 'gen...",Enkhmurun Erdenebat,537189683968489,"{'lower_bound': '20000', 'upper_bound': '24999'}",100179472105159,Democratic Party Ардчилсан нам,{'lower_bound': '1000001'},"['facebook', 'instagram']","[{'percentage': '0.010928', 'region': 'Arkhang...","{'lower_bound': '0', 'upper_bound': '99'}",nan,nan
3,2021-04-29,🇲🇳Монгол улсын #Ерөнxийлөгчид Ардчилсан намаас...,nan,2021-04-29,2021-05-02,https://www.facebook.com/ads/archive/render_ad...,USD,"[{'percentage': '1.0E-5', 'age': '65+', 'gende...",Enkhmurun Erdenebat,246035923936971,"{'lower_bound': '350000', 'upper_bound': '3999...",100179472105159,Democratic Party Ардчилсан нам,{'lower_bound': '1000001'},"['facebook', 'instagram']","[{'percentage': '0.011469', 'region': 'Arkhang...","{'lower_bound': '100', 'upper_bound': '199'}",nan,nan
4,2021-04-29,АРДЧИЛСАН НАМЫН ЭЭЛЖИТ XII ЦАХИМ ИХ ХУРАЛ 2021...,nan,2021-04-29,2021-05-03,https://www.facebook.com/ads/archive/render_ad...,USD,"[{'percentage': '3.6E-5', 'age': '18-24', 'gen...",Enkhmurun Erdenebat,170373828293644,"{'lower_bound': '90000', 'upper_bound': '99999'}",100179472105159,Democratic Party Ардчилсан нам,{'lower_bound': '1000001'},"['facebook', 'instagram']","[{'percentage': '1', 'region': 'Ulan Bator'}]","{'lower_bound': '100', 'upper_bound': '199'}",nan,nan


In [9]:
pages.head()

,Page ID,Page Name,Disclaimer,Amount Spent (USD),Number of Ads in Library
0,100179472105159,Democratic Party Ардчилсан нам,Enkhmurun Erdenebat,1710,32
1,844579789004530,Хассуурийн Ганхуяг,Khassuuri Gankhuyag,1530,45
2,108721589821589,Ганибалын Амартүвшин,Nurbyek Samalkhan,472,36
3,1204136139605142,Болдын Жавхлан Javkhlan Bold,Enkhtungalag Nyamdorj,458,7
4,1645028429160586,Дамбын Батлут,Энхчулуун Чинбат,415,14


## Data Cleaning

### Date Filter

Our time period of interest is April 2021. Filter the ads dataframe on that. 

In [10]:
ads['ad_delivery_start_time'] = pd.to_datetime(ads['ad_delivery_start_time'])
ads['ad_delivery_stop_time'] = pd.to_datetime(ads['ad_delivery_stop_time'])
ads['ad_creation_time'] = pd.to_datetime(ads['ad_creation_time'])

In [11]:
ads = ads[(ads['ad_delivery_start_time'] >= start_date) & (ads['ad_delivery_start_time'] <= end_date)]

### Eliminate Duplicates

The Facebook API has duplicate pages that have the same Page ID but different numbers of ads. It seem the pages farther up the list are the newer/correct ones, so we will keep those and drop the duplicates.

In [12]:
pages[pages.duplicated(subset=['Page ID'], keep=False)].sort_values('Page Name').head()

,Page ID,Page Name,Disclaimer,Amount Spent (USD),Number of Ads in Library
500,125084321493311,Altansor Tsogtkhuu,Munkhtur Ravjir,≤100,1
90,125084321493311,Altansor Tsogtkhuu,Altansor Soroo,385,10
621,125084321493311,Altansor Tsogtkhuu,These ads ran without a disclaimer,≤100,1
437,1118243471545672,Baatar Jamsran Жамсрангийн Баатар,Tamir Odgerel,≤100,3
242,1118243471545672,Baatar Jamsran Жамсрангийн Баатар,Sodmagnai Enkhjargal,≤100,2


In [13]:
pages = pages.drop_duplicates(subset=['Page ID'], keep='first')

In [14]:
pages.head()

,Page ID,Page Name,Disclaimer,Amount Spent (USD),Number of Ads in Library
0,100179472105159,Democratic Party Ардчилсан нам,Enkhmurun Erdenebat,1710,32
1,844579789004530,Хассуурийн Ганхуяг,Khassuuri Gankhuyag,1530,45
2,108721589821589,Ганибалын Амартүвшин,Nurbyek Samalkhan,472,36
3,1204136139605142,Болдын Жавхлан Javkhlan Bold,Enkhtungalag Nyamdorj,458,7
4,1645028429160586,Дамбын Батлут,Энхчулуун Чинбат,415,14


For page spend we will list everything '<=100' to 100 for proper plotting.

In [15]:
pages['Amount Spent (USD)'] = pages['Amount Spent (USD)'].replace(to_replace='≤100', value='100').astype('int')

Now that we have removed the duplicates we can merge the dataframes. 

In [16]:
df = ads.merge(pages, how='left', left_on='page_id', right_on='Page ID')

In [17]:
df = df.drop(columns=['Amount Spent (USD)','Number of Ads in Library'])

In [18]:
df.sample(2)

,ad_creation_time,ad_creative_body,ad_creative_link_title,ad_delivery_start_time,ad_delivery_stop_time,ad_snapshot_url,currency,demographic_distribution,funding_entity,id,impressions,page_id,page_name,potential_reach,publisher_platforms,region_distribution,spend,ad_creative_link_caption,ad_creative_link_description,Page ID,Page Name,Disclaimer
207,2021-04-18,🏙 Энэ хөл хориотой үед Сөүл Гарден цогцолборын...,nan,2021-04-18,2021-04-19,https://www.facebook.com/ads/archive/render_ad...,USD,"[{'percentage': '0.000364', 'age': '35-44', 'g...",nan,299950818168566,"{'lower_bound': '10000', 'upper_bound': '14999'}",577689219250430,Seoul Garden цогцолбор,{'lower_bound': '1000001'},"['facebook', 'instagram']","[{'percentage': '1', 'region': 'Ulan Bator'}]","{'lower_bound': '0', 'upper_bound': '99'}",nan,nan,577689219250430,Seoul Garden цогцолбор,These ads ran without a disclaimer
27,2021-04-15,"Засгийн Газрын хэрэгжүүлэгч агентлаг Гэр бүл, ...",nan,2021-04-15,2021-04-17,https://www.facebook.com/ads/archive/render_ad...,USD,"[{'percentage': '0.034195', 'age': '18-24', 'g...",Nurbyek Samalkhan,1020541158479190,"{'lower_bound': '3000', 'upper_bound': '3999'}",108721589821589,Ганибалын Амартүвшин,{'lower_bound': '1000001'},['facebook'],"[{'percentage': '1', 'region': 'Ulan Bator'}]","{'lower_bound': '0', 'upper_bound': '99'}",nan,nan,108721589821589,Ганибалын Амартүвшин,Nurbyek Samalkhan


### Convert nested data to columns

**Potential Reach**

In [19]:
df = pd.concat([df.drop(columns='potential_reach'), 
                pd.json_normalize(df['potential_reach'].map(eval))], axis=1)

In [20]:
df = df.rename(columns={'lower_bound':'reach_lower', 'upper_bound':'reach_upper'})

**Spend**

In [21]:
df = pd.concat([df.drop(columns='spend'), 
                pd.json_normalize(df['spend'].map(eval))], axis=1)

In [22]:
df = df.rename(columns={'lower_bound':'spend_lower', 'upper_bound':'spend_upper'})

**Impressions**

In [23]:
df = pd.concat([df.drop(columns='impressions'), 
                pd.json_normalize(df['impressions'].map(eval))], axis=1)

In [24]:
df = df.rename(columns={'lower_bound':'impressions_lower', 
                        'upper_bound':'impressions_upper'})

### Separate Demographics into Columns

In [25]:
pd.DataFrame(ast.literal_eval(df['demographic_distribution'].iloc[0]))

,percentage,age,gender
0,0.046576,55-64,male
1,0.108474,25-34,female
2,4.1E-5,55-64,unknown
3,0.019407,65+,male
4,0.10974,45-54,male
5,0.028681,18-24,male
6,0.052296,55-64,female
7,0.122732,35-44,female
8,0.10018,45-54,female
9,0.180667,25-34,male


### Filter to only Mongolian ads

There are often international advertisers that will end up in the political and social issue ad category. These ads are generally in English or Russian. We can filter these ads out by detecting the language of each ad. We will then filter for only Mongolian.

In [26]:
def translate_text(target, text):
    """Translates text into the target language.

    Target must be an ISO 639-1 language code.
    See https://g.co/cloud/translate/v2/translate-reference#supported_languages
    """
    import six
    from google.cloud import translate_v2 as translate

    translate_client = translate.Client()

    if isinstance(text, six.binary_type):
        text = text.decode("utf-8")

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.translate(text, target_language=target)

    print(u"Text: {}".format(result["input"]))
    print(u"Translation: {}".format(result["translatedText"]))
    print(u"Detected source language: {}".format(result["detectedSourceLanguage"]))

In [28]:
df['ad_creative_body'][0]

'Монгол улсын ерөнхийлөгчид Ардчилсан намаас нэр дэвшигчдийн бодлогын мэтгэлцээн'

In [29]:
translate_text('en', df['ad_creative_body'][0])

DefaultCredentialsError: Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or explicitly create credentials and re-run the application. For more information, please see https://cloud.google.com/docs/authentication/getting-started

In [38]:
df['ad_lang'] = df['ad_creative_body'].apply(lambda x: TextBlob(x).detect_language())

HTTPError: HTTP Error 429: Too Many Requests

In [ ]:
df = df[df['ad_lang'] == 'mn']

## Top Facebook pages by ad spending

Calculate the number of days the ad ran. This sets a minimum value for ad spend. 

In [109]:
df['ad_days'] = df['ad_delivery_stop_time'] - df['ad_delivery_start_time']

In [110]:
df['ad_days'] = df['ad_days'].dt.days

Convert spend_lower and spend_upper to int so we can do math on it.

In [111]:
df['spend_lower'] = df['spend_lower'].astype('int')

In [112]:
df['spend_upper'] = df['spend_upper'].astype('int')

If the spend_lower value is 0, we can change that to a realistic minimum by taking the number of days as the absolute minimum, as there is a 1 dollar per day minimum. 

In [118]:
df['spend_lower'] = df['spend_lower'].mask(df['spend_lower']==0, df['spend_lower'] + df['ad_days'])

In [119]:
df.head(2)

,ad_creation_time,ad_creative_body,ad_creative_link_title,ad_delivery_start_time,ad_delivery_stop_time,ad_snapshot_url,currency,demographic_distribution,funding_entity,id,impressions,page_id,page_name,publisher_platforms,region_distribution,ad_creative_link_caption,ad_creative_link_description,Page ID,Page Name,Disclaimer,reach_lower,reach_upper,spend_lower,spend_upper,ad_days
0,2021-04-30,Монгол улсын ерөнхийлөгчид Ардчилсан намаас нэ...,Монгол улсын ерөнхийлөгчид Ардчилсан намаас нэ...,2021-04-30,2021-05-02,https://www.facebook.com/ads/archive/render_ad...,USD,"[{'percentage': '0.046576', 'age': '55-64', 'g...",Enkhmurun Erdenebat,808086169839504,"{'lower_bound': '30000', 'upper_bound': '34999'}",100179472105159,Democratic Party Ардчилсан нам,['facebook'],"[{'percentage': '0.014488', 'region': 'Övörkha...",nan,nan,100179472105159,Democratic Party Ардчилсан нам,Enkhmurun Erdenebat,1000001,NaN,2.0,99,2.0
1,2021-04-29,САНАЛ ӨГӨХ ЗААВАР:🇲🇳nАрдчилсан нам Монгол улса...,nan,2021-04-30,2021-05-02,https://www.facebook.com/ads/archive/render_ad...,USD,"[{'percentage': '0.000132', 'age': '25-34', 'g...",Enkhmurun Erdenebat,811746283053829,"{'lower_bound': '25000', 'upper_bound': '29999'}",100179472105159,Democratic Party Ардчилсан нам,"['facebook', 'instagram']","[{'percentage': '0.012833', 'region': 'Arkhang...",nan,nan,100179472105159,Democratic Party Ардчилсан нам,Enkhmurun Erdenebat,1000001,NaN,2.0,99,2.0


In [121]:
df.groupby(by=['Page Name']).sum().sort_values('spend_upper', ascending=False)[:10]

,spend_lower,spend_upper,ad_days
Page Name,,,
Humane Society International,50000.0,57491,24.0
EveryJew,10710.0,13089,23.0
РЫБАКОВ ФОНД,8000.0,8999,2.0
Speechify,6002.0,7098,2.0
Asian Development Bank Institute - ADBI,1600.0,2198,11.0
Баярсайхан Галигаа Bayarsaikhan Galigaa,1301.0,1894,21.0
Braindom 2: Riddle Game,1000.0,1499,0.0
ProstaGenix.com,1000.0,1499,7.0
Кулинарная школа ЩиБорщи,1000.0,1499,2.0


## Top Payers

In [123]:
df.groupby(by=['Disclaimer']).sum().sort_values('spend_upper', ascending=False)[:10]

,spend_lower,spend_upper,ad_days
Disclaimer,,,
These ads ran without a disclaimer,85157.0,109368,318.0
Баярсайхан Галигаа,1301.0,1894,21.0
Khassuuri Gankhuyag,42.0,1188,42.0
Enkhmurun Erdenebat,309.0,1092,19.0
Batsaikhan Bat,44.0,1089,44.0
Nurbyek Samalkhan,31.0,990,31.0
Энхчулуун Чинбат,15.0,990,15.0
Baska Mn,41.0,891,41.0
Уянгаа Алтангэрэл,24.0,891,24.0


In [137]:
df['page_id'] = df['page_id'].astype('int')

In [138]:
df.groupby(by=['Disclaimer','Page Name']).mean()

page_id  \
Disclaimer                         Page Name                                                          
Anar Munkhsaikhan                  Ламжавын Гүндалай                                   3.032196e+14   
Anudari Ayush                      Women for Change NGO / Өөрчлөлтийн төлөөх Эмэгт...  1.550179e+14   
Baagii Bat-Erdene                  Монгол Улсын Сонгуулийн ерөнхий хороо               1.085788e+14   
Baasanjargal Baska                 Нэхүүл Нам Бус Залуусын Хөдөлгөөн                   2.850413e+14   
Baasanjav Damdinsuren              Бямбасүрэнгийн Мягмар                               5.607064e+14   
Baska Mn                           Todmedee.mn                                         4.579059e+14   
                                   Б.Жаргалсайхан                                      7.429019e+14   
                                   Дуулиа бичлэгүүд                                    1.005837e+14   
                                   Мэдээ мэдээлэл Төв                                  7.214332e+14   
                                   Сенса мэдээ                                         1.074604e+14   
                                   Сошиал Мэдээ                                        4.023803e+14   
Batsaikhan Bat                     Niitlel.mn нийтлэл.мн                               1.004319e+14   
                                   SNEWS                                               5.808821e+14   
                                   Сонин мэдээ                                         1.604427e+14   
Batzaya Otgon                      Vip76                                               5.172839e+14   
Chantsaldulam Munkhtumur           Ардчилсан хувьсгалын ахмад зүтгэлтний хороо         1.942350e+15   
Delgerekh Khishi                   Өлзий-Оршихын Сумъяабаатар                          6.722139e+14   
Dulguun Enkhtsogt                  LEAD Alumni Association                             2.065952e+15   
Enhbadrah Bathuyag                 УИХ-ын гишүүн С.Батболдын Ажлын Алба                3.458816e+14   
Enkh-Amgalan Byambasuren           Бямбасүрэнгийн Энх-Амгалан Enkh-Amgalan Byambas...  1.086766e+15   
Enkhkhudulmur Tsogerdene           Ардчилсан нам                                       2.101752e+14   
Enkhmurun Erdenebat                Democratic Party Ардчилсан нам                      1.001795e+14   
Enkhtungalag Nyamdorj              Болдын Жавхлан Javkhlan Bold                        1.204136e+15   
Ganbaatar Jambal                   Жамбалын Ганбаатар                                  1.715214e+14   
Gantulga Donorov                   Доноровын Гантулга - Gantulga Donorov               7.173680e+14   
Gantulga Lkhaakhuu                 Судлаач Ганаа                                       1.583373e+14   
Hunnu Shoes Mongolian Brand        Hunnu Shoes Mongolian Brand                         1.750758e+15   
Jargalsaikhan Derem                Дархан-Уул аймаг дахь Ардчилсан нам                 1.015596e+14   
Khassuuri Gankhuyag                Хассуурийн Ганхуяг                                  8.445798e+14   
Lamjav Zolbaatar                   Barometer                                           2.555424e+14   
MUNKHBAATAR Lkhagva                MUNKHBAATAR Lkhagva                                 2.847325e+14   
Munkhchimeg Moonon                 Н.Алтаншагай                                        6.927627e+14   
Munkhtur Ravjir                    Тогмидын Доржханд                                   1.486822e+15   
                                   Хөдөлмөрийн Үндэсний Нам - ХҮН                      1.715979e+15   
Naranbaatar Nanzad                 Нанзадын Наранбаатар                                8.461313e+14   
Nurbyek Samalkhan                  Ганибалын Амартүвшин                                1.087216e+14   
Ochirpurev Nyamjav                 Бямбацогт Сандаг                                    3.194268e+14   
Purevdorj Bukhchuluun              Purevdorj Bukhchuluun                   

If we need to convert a list inside a series to a dataframe we can do this. We will use this later. 

In [ ]:
pd.DataFrame(ast.literal_eval(df['demographic_distribution'][0]))